## General

In [91]:
# Import libraries
import string
import random
import numpy as np
import networkx as nx
from numpy import linalg as LA
import matplotlib.pyplot as plt

## Load Map

In [26]:
# Load map
map_ = np.loadtxt("simple-map-dungeon.txt").astype(int)

## Graph Creation (NetworkX)

In [21]:
# Create graph instance
G = nx.Graph()

# Populate graph by creating
# edges between neighbours
for y in range(map_.shape[0]):
    for x in range(map_.shape[1]):
        # Skip iteration if obstacle
        if map_[y,x]:
            continue
        
        # Add edge to non-obstacle element to the right
        if y+1 < map_.shape[0] and map_[y+1,x] == 0:
            G.add_edge((x,y),(x,y+1))
        
        # Add edge to non-obstacle element to the left
        if y-1 >= 0 and  map_[y-1,x] == 0:
            G.add_edge((x,y),(x,y-1))
        
        # Add edge to non-obstacle element to the bottom
        if x+1 < map_.shape[1] and  map_[y,x+1] == 0:
            G.add_edge((x,y),(x+1,y))
        
        # Add edge to non-obstacle element to the top
        if x-1 >= 0 and  map_[y,x-1] == 0:
            G.add_edge((x,y),(x-1,y))

## Source-Target Distances

In [44]:
# Compute dictionary of distances
# between any node and any other one
D = nx.shortest_path(G)

## K-medoids clustering

In [59]:
# Global variables
K = 4
TMAX = 50

# Medoids list
medoids = []

# Pick randomly medoids
# among the nodes of the
# graph
for i in range(K):
    medoids.append(random.choice(list(G.nodes())))

# Itareate over tmax
for epoch in range(TMAX):
    # Create a list of sets
    # for each medoid selected
    clusters = [set() for _ in range(K)]
    
    # Populate clusters for
    # each medoid set
    for node in G.nodes():
        # Temp variables to
        # keep track of closest
        # medoids
        min_distance = np.inf
        closest_medoid = None
        
        # Iterate over medoids
        for medoid in medoids:
            # Fetch shortest path distance
            distance = len(D[node][medoid])
            
            # Update min distance
            # and respective closest
            # medoid
            if distance < min_distance:
                min_distance = distance
                closest_medoid = medoid
        
        # Add vertex to closest cluster
        idx = medoids.index(closest_medoid)
        clusters[idx].add(node)

    # Update medoid based on updated clusters
    for idx, clust_set in enumerate(clusters):
        # Temp variables
        min_distance = np.inf
        closest_medoid = None
        
        # Iterate over ith cluster
        for node_j in clust_set:
            # Compute sum distance for each
            # cluster points with respect to
            # all other points in the cluster
            sum_distance = sum(len(D[node_j][node_l]) for node_l in clust_set)
            
            # Update min distance and
            # respective best medoid
            if sum_distance < min_distance:
                min_dist = sum_distance
                closest_medoid = node_j
        
        # Update medoid
        medoids[idx] = closest_medoid  

## Waypoint Graph 

In [141]:
map_clustered = map_.copy()
for y in range(map_.shape[0]):
    for x in range(map_.shape[1]):
        node = map_[y,x]
        if node == 0:
            for idx, clust_set in enumerate(clusters):
                if (x,y) in clust_set:
                    map_clustered[y, x] = idx + 1
        else:
            map_clustered[y, x] = 0
map_clustered 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0],
       [0, 4, 4, 4, 4, 4,

## Spectral Clustering

In [94]:
# Compute adjacency matrix
A = nx.adjacency_matrix(G).todense()

# Create degree matrix
D = np.zeros((A.shape[0], A.shape[1]))

# Populate degree matrix
for y in range(A.shape[0]):
    degree = 0
    for x in range(A.shape[1]):
        degree += A[y,x]
    
    # Update degree for node y
    D[y, y] = degree

# Compute the laplacian graph
L = D - A

# Perform EigenValue Decomposition
w, v = LA.eigh(L)

In [96]:
v

matrix([[ 5.36828127e-02, -1.14978158e-01,  9.00154573e-02, ...,
          2.65689126e-12, -4.87964724e-13,  1.05290375e-13],
        [ 5.36828127e-02, -1.15094983e-01,  9.05864754e-02, ...,
         -7.58792414e-12,  1.35923412e-12, -2.89349177e-13],
        [ 5.36828127e-02, -1.14669395e-01,  8.85331693e-02, ...,
         -7.60035828e-12,  1.48833065e-12, -3.32379532e-13],
        ...,
        [ 5.36828127e-02,  3.62346349e-02,  2.40690836e-02, ...,
         -1.01308840e-02, -6.59734268e-03, -3.93913601e-03],
        [ 5.36828127e-02,  3.61467023e-02,  2.36887111e-02, ...,
          1.39954499e-02,  9.78520492e-03,  6.23824652e-03],
        [ 5.36828127e-02,  3.60443722e-02,  2.32490132e-02, ...,
         -8.94612833e-03, -6.96779801e-03, -4.87295844e-03]])